# Schema Usage

The schema does not actually store any data.  Instead, it is an interface which allows us to interact with numpy/torch tensors in a semantic manner.  It lets us convert between storage vectors (i.e. how we store the building parameters numerically on disk), simulation objects (e.g. Archetypal Templates and PyUmi Shoeboxes) and machine learning model imports (i.e. torch tensors with full hourly schedule data).

## Notebook setup

We need some jank to get relative imports working.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np

## Initialize the Schema

In [2]:
from schema import Schema, ShoeboxGeometryParameter
schema = Schema()

c:\anacondas\envs\ml-for-bem-sampling\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.2, the latest is 0.3.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


Let's see what's in the schema:

In [3]:
schema.parameter_names

['id',
 'base_template',
 'base_epw',
 'width',
 'height',
 'facade_2_footprint',
 'perim_2_footprint',
 'roof_2_footprint',
 'footprint_2_ground',
 'shading_fact',
 'wwr_n',
 'wwr_e',
 'wwr_s',
 'wwr_w',
 'orientation',
 'LightingPowerDensity',
 'EquipmentPowerDensity',
 'PeopleDensity',
 'FacadeRValue',
 'RoofRValue',
 'PartitionRValue',
 'SlabRValue',
 'schedules_seed',
 'schedules']

We can access a schema parameter from the schema with list indexing:

In [5]:
print(schema["width"])
print(schema["schedules"])
print(schema["orientation"])

---width---
shape_storage=(1,), shape_ml=(1,), dtype=scalar
Width [m]
---schedules---
shape_storage=(8, 16), shape_ml=(8, 8760), dtype=matrix
A matrix in the storage vector with operations to apply to schedules; a matrix of timeseries in ml vector
---orientation---
shape_storage=(1,), shape_ml=(4,), dtype=onehot
Shoebox Orientation


We see that each parameter may have multiple different lengths in the storage vector and ML vector.

We can also print a summary of the whole schema:

In [8]:
print(schema)

-------- Schema --------
---- id ----
shape storage: (1,) / shape ml: (0,)
location storage: 0->1 / location ml: 0->0

---- base_template ----
shape storage: (1,) / shape ml: (0,)
location storage: 1->2 / location ml: 0->0

---- base_epw ----
shape storage: (1,) / shape ml: (0,)
location storage: 2->3 / location ml: 0->0

---- width ----
shape storage: (1,) / shape ml: (1,)
location storage: 3->4 / location ml: 0->1

---- height ----
shape storage: (1,) / shape ml: (1,)
location storage: 4->5 / location ml: 1->2

---- facade_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 5->6 / location ml: 2->3

---- perim_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 6->7 / location ml: 3->4

---- roof_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 7->8 / location ml: 4->5

---- footprint_2_ground ----
shape storage: (1,) / shape ml: (1,)
location storage: 8->9 / location ml: 5->6

---- shading_fact ----
shape storage: (1,) /

We see that the length of the storage vector is significantly smaller than the length the vector the ML model will see.

## Generating new design vectors in storage space

First let's generate a new, empty design vector, and update the Roof R-Value, and then check that it updated correctly:

In [9]:
storage_vector = schema.generate_empty_storage_vector()
schema.update_storage_vector(storage_vector=storage_vector, parameter="RoofRValue", value=25)
schema["RoofRValue"].extract_storage_values(storage_vector)

25.0

If we print out the full vector, we should be able to see th 25 and a whole bunch of zeros:

In [10]:
print(storage_vector)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 25.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


Let's create a new batch of designs:

In [15]:
batch_size = 20
storage_batch = schema.generate_empty_storage_batch(batch_size)
storage_batch.shape

(20, 151)

Great, we see that it has 20 design vectors with 151 values each.

Let's try updating all of the facade R-values values in a batch with the same value:

In [19]:
schema.update_storage_batch(storage_batch, parameter="FacadeRValue", value=14)
schema["FacadeRValue"].extract_storage_values_batch(storage_batch)

array([[14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.]])

Now let's try updating an entire batch with random values.  We can also unnormalize the uniform random variable into the desired range:

In [17]:
parameter = "SlabRValue"
n = batch_size
shape = (n, *schema[parameter].shape_storage)
values = np.random.rand(*shape) # create a random sample with appropriate shape
values = schema[parameter].unnormalize(values) # schema parameter must be a numeric type with min/max defined for unnormalize to work
schema.update_storage_batch(storage_batch, parameter=parameter, value=values)
schema[parameter].extract_storage_values_batch(storage_batch)

array([[49.69044519],
       [35.83417614],
       [36.15324811],
       [39.70141417],
       [36.91452215],
       [35.8058487 ],
       [47.14623408],
       [28.00483113],
       [11.64489202],
       [ 5.76324733],
       [43.97896559],
       [23.8966386 ],
       [18.18477858],
       [22.40312448],
       [24.79133712],
       [48.51735346],
       [23.83342887],
       [40.33448286],
       [41.80882116],
       [37.35796057]])

Finally, let's try updating just a subset of the batch by using the `index` parameter:

*nb: we can also use an int instead of a tuple for `index` to only update a single vector's parameter*

In [18]:
start = 2
n = 8
end = start + n
parameter = "PartitionRValue"
shape = (n, *schema[parameter].shape_storage)
values = np.random.rand(*shape) # create a random sample with appropriate shape

schema.update_storage_batch(storage_batch, index=(start,end), parameter=parameter, value=values)
schema[parameter].extract_storage_values_batch(storage_batch) 

array([[0.        ],
       [0.        ],
       [0.14358503],
       [0.41262764],
       [0.5909276 ],
       [0.74166519],
       [0.25399637],
       [0.09774832],
       [0.59075003],
       [0.8960576 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

A useful technique will be to start with a small batch, and then duplicate it in concatenations along `axis=0` as we build up our mixed grid/hypercube/random samples.  Let's start by creating a new batch with a single vector.

In [38]:
storage_batch = schema.generate_empty_storage_batch(1)
storage_batch.shape

(1, 151)

Now let's say some baseline parameters (e.g. pulled from ResStock)

In [39]:
schema.update_storage_batch(storage_batch, parameter="FacadeRValue", value=20)
schema.update_storage_batch(storage_batch, parameter="RoofRValue", value=30)
schema.update_storage_batch(storage_batch, parameter="LightingPowerDensity", value=7.2)
schema.update_storage_batch(storage_batch, parameter="base_template", value=15)
storage_batch = np.concatenate([storage_batch for _ in range(4)], axis=0)
storage_batch

array([[ 0. , 15. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  7.2,  0. ,  0. , 20. , 30. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , 

Now let's set the orientations:

In [40]:
values = np.arange(4).reshape(-1,1)
parameter = "orientation"
schema.update_storage_batch(storage_batch, parameter=parameter, value=values)
schema[parameter].extract_storage_values_batch(storage_batch)

array([[0.],
       [1.],
       [2.],
       [3.]])

Looks good!  Now let's stack this up and begin generating some geometric variations.

In [41]:
orientations_per_base = 4
geometric_variations_per_orientation = 5

In [42]:
storage_batch = np.repeat(storage_batch, geometric_variations_per_orientation, axis=0)
storage_batch.shape

(20, 151)

In [45]:
schema["orientation"].extract_storage_values_batch(storage_batch)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.]])

Looks good!  let's start populating this: if we wanted to use repeating values, we could do nested loops:

In [51]:
for i in range(orientations_per_base):
	n = geometric_variations_per_orientation # how many design vectors in this mini batch
	start = i*n # where this mini batch starts in the parent batch
	end = start + n # where this mini batch ends in the parent batch
	for j,parameter in enumerate(schema.parameters):
		if isinstance(parameter, ShoeboxGeometryParameter):
			name = parameter.name
			shape = parameter.shape_storage
			np.random.seed(j+20923) # arbitrary but reliable seed
			values = np.random.rand(n, *shape) 
			values = parameter.unnormalize(values)
			schema.update_storage_batch(storage_batch, index=(start,end), parameter=name, value=values)


In [52]:
schema["wwr_e"].extract_storage_values_batch(storage_batch)

array([[0.73061709],
       [0.49898089],
       [0.93358064],
       [0.31829062],
       [0.86278618],
       [0.73061709],
       [0.49898089],
       [0.93358064],
       [0.31829062],
       [0.86278618],
       [0.73061709],
       [0.49898089],
       [0.93358064],
       [0.31829062],
       [0.86278618],
       [0.73061709],
       [0.49898089],
       [0.93358064],
       [0.31829062],
       [0.86278618]])

In [54]:
schema["width"].extract_storage_values_batch(storage_batch)

array([[2.47060375],
       [1.55185543],
       [3.44850939],
       [4.77931689],
       [2.03592266],
       [2.47060375],
       [1.55185543],
       [3.44850939],
       [4.77931689],
       [2.03592266],
       [2.47060375],
       [1.55185543],
       [3.44850939],
       [4.77931689],
       [2.03592266],
       [2.47060375],
       [1.55185543],
       [3.44850939],
       [4.77931689],
       [2.03592266]])

Great, these are repeating correctly!  Now, suppose we want to just slightly perturb all of these so that they aren't perfectly repeating, but are close to repeating:

In [56]:
for i,parameter in enumerate(schema.parameters):
	n = storage_batch.shape[0]
	name = parameter.name
	shape = parameter.shape_storage
	perturbations = np.random.rand(n,*shape)*0.2 - 0.1
	values = parameter.extract_storage_values_batch(storage_batch)
	values += perturbations
	schema.update_storage_batch(storage_batch,parameter=name,value=values)

schema["width"].extract_storage_values_batch(storage_batch)

array([[2.51081291],
       [1.46387821],
       [3.36970034],
       [4.76810012],
       [1.99254176],
       [2.54458022],
       [1.59163977],
       [3.52996215],
       [4.87626705],
       [2.06258373],
       [2.3725389 ],
       [1.60041238],
       [3.47399806],
       [4.74630787],
       [2.05948589],
       [2.42965623],
       [1.51820695],
       [3.54316544],
       [4.81372785],
       [2.06793996]])

Great!  We see that they are close to their previous values, but not identical.  

Alternatively, we might prefer to simply use fully random geometric variations for all of our orientation duplicates, rather than repeating the geometry across orientations:

In [57]:
for i,parameter in enumerate(schema.parameters):
	n = storage_batch.shape[0]
	name = parameter.name
	shape = parameter.shape_storage
	values = np.random.rand(n,*shape)
	values = parameter.unnormalize(values)
	schema.update_storage_batch(storage_batch,parameter=name,value=values)

schema["width"].extract_storage_values_batch(storage_batch)

array([[4.13487261],
       [3.97507724],
       [3.48762092],
       [3.2486845 ],
       [3.60323479],
       [3.94003224],
       [4.95712946],
       [2.9570808 ],
       [2.20389045],
       [1.92241243],
       [4.72837206],
       [1.8059675 ],
       [3.92463522],
       [1.67167951],
       [1.89828836],
       [3.84975856],
       [4.77331265],
       [4.97268823],
       [2.33213755],
       [2.10742535]])